In [5]:
# Scale LSP rasters from MODIS EVI
# Current raster band values are (TIMESAT) index based 
# from this we change it to actual day of the year values
# for example the scaled raster image of SOS having pixel value of 5 means 
# the start of the growing year occured on January 5

import numpy as np
import pandas as pd
import rioxarray as rxr
import rasterio as rio
import xarray as xr
import matplotlib.pyplot as plt
import os
import glob

In [6]:
#Load Unscaled Raster 

unscaled_raster_path = r"..\Data\Unscaled_LSP_Rasters"
scaled_raster_path  = r"..\Data\Scaled_LSP_Rasters"
LSP_metric = "sos"
start_year = 2001
end_year = 2024

var_path = os.path.join(unscaled_raster_path, LSP_metric)
var_files = sorted(glob.glob(os.path.join(var_path, "*.tif")))
var_dates = [pd.to_datetime(("20"+ os.path.basename(f)[-6:-4] ), format='%Y') for f in var_files]

var_raster_list = []
for f in var_files:
    r = rxr.open_rasterio(f, chunks={'x': 1024, 'y': 1024}, masked=True)
    #r_matched = r.rio.reproject_match(sos01)
    var_raster_list.append(r)

var_rasters = xr.concat(var_raster_list, dim=pd.Index(var_dates, name='time'), join='exact')

# del var_raster_list, r, r_matched, 


In [7]:
sos01 = var_rasters.sel(time = '2001-01-01')


In [ ]:
#Scale SOS Raster first

for year in range(start_year, end_year-15):
    i = year - start_year
    year_dt = pd.to_datetime(str(year), format='%Y')
    file_to_scale = var_rasters.sel(time=year_dt)

    
    valid_data = file_to_scale.where(file_to_scale > 0)
    
    
    int_part = np.floor(valid_data)
    dec_part = valid_data - int_part

    int_scaled = int_part - (23*i)
    int_scaled2 = 1 + ((int_scaled - 1) * 16)

    dec_scaled = dec_part * 16
    scaled_value = int_scaled2 + dec_scaled

    
    scaled_data = scaled_value.where(file_to_scale > 0, -999) 
    scaled_data.rio.to_raster(os.path.join(scaled_raster_path, f"{LSP_metric}", f"{LSP_metric}{year}.tif"))


    
    